In [5]:
import numpy as np
from typing import List, Tuple
from os import path, getcwd
import random

(a) Implement a variant of this algorithm based on memetic algorithms (MAs). Use
the 2-opt algorithm as local search technique in the memetic algorithm. The
2-opt algorithm tries to swap all pairs of cities to see if this improves the length
of the tour (see, e.g. https://en.wikipedia.org/wiki/2-opt).

In [6]:
cities_path = path.join(getcwd(), "file-tsp.txt")

candidate: List[int]

def eval_fitness(x, distance):
    return 1/distance

def get_cities():
    return np.genfromtxt(cities_path)

In [16]:
def order_for_search(x:List[int], cut_point1:int, cut_point2:int) -> List[int]:
    """
    HELPER-FUNCTION: Orders the phenotype around the crossover points in order to simplify the merging.
    """
    return np.concatenate((x[cut_point2:], x[0:cut_point2]))

def produce_offspring(search_dig:List[int], cut_seq:List[int], cut_points:Tuple[int,int], phen_length:int) -> List[int]:
    offspring = np.zeros(phen_length)
    p1, p2 = cut_points
    offspring[p1:p2] = cut_seq
    for i in range(p2, phen_length):
        for digit in search_dig:
            if not digit in offspring:
                offspring[i] = digit
                break
    for i in range(0, p1):
        for digit in search_dig:
            if not digit in offspring:
                offspring[i] = digit
                break
    return offspring

def crossover(candidate1:List[int], candidate2: List[int]) -> Tuple[List[int], List[int]]:
    cut_point1 = random.randint(0, len(candidate1)-1)
    cut_point2 = random.randint(cut_point1, len(candidate1))
    seq_1 = candidate1[cut_point1:cut_point2]
    seq_2 = candidate2[cut_point1:cut_point2]

    parent1_search_dig = order_for_search(candidate1, cut_point1, cut_point2)
    parent2_search_dig = order_for_search(candidate2, cut_point1, cut_point2)

    phen_length = len(candidate1)

    offspring_1 = produce_offspring(parent2_search_dig, seq_1, (cut_point1, cut_point2), phen_length)
    offspring_2 = produce_offspring(parent1_search_dig, seq_2, (cut_point1, cut_point2), phen_length)

    return (offspring_1, offspring_2)

crossover(np.array([3,5,7,2,1,6,4,8]), np.array([2,5,7,6,8,1,3,4]))

[]
[]
[5 7 6 8 1 3 4 2]


(array([2., 5., 7., 6., 8., 1., 3., 4.]),
 array([3., 5., 7., 2., 1., 6., 4., 8.]))

In [18]:
def mutate(candidate:List[int]) -> List[int]:
    pos1 = random.randint(0, len(candidate))
    pos2 = random.randint(0, len(candidate))
    while pos1 == pos2:
        pos2 = random.randint(0, len(candidate))

    mutated = candidate.copy()
    mutated[pos1] = candidate[pos2]
    mutated[pos2] = candidate[pos1]

    return mutated

#mutate([5,8,7,2,1,6,3,4])

[5, 8, 6, 2, 1, 7, 3, 4]

In [37]:
def init_population(size:int, chromosomes:int):
    population = np.zeros((size, chromosomes), dtype="int")
    val_range = np.arange(chromosomes)
    for i in range(size):
        np.random.shuffle(val_range)
        population[i] = val_range
    
    return population
        

array([[0, 3, 2, 1, 4],
       [1, 4, 0, 3, 2],
       [0, 2, 1, 4, 3],
       [2, 3, 4, 0, 1],
       [1, 3, 0, 2, 4],
       [4, 3, 2, 1, 0],
       [4, 3, 0, 2, 1],
       [3, 2, 4, 0, 1],
       [1, 3, 0, 2, 4],
       [3, 2, 0, 1, 4]])

In [ ]:
def tournament_select(population:np.ndarray):
    """Returns the index of the candidate with the highest fitness. Does binary tournament selection"""
    cand1 = random.randint(0, len(population))
    cand2 = random.randint(0, len(population))
    while cand1 == cand2:
        cand2 = random.randint(0, len(population))
    return np.argmax(eval_fitness(cand1), eval_fitness(cand2))


In [46]:
def two_opt_swap(phen:List[int], i:int, k:int) -> List[int]:
    new_route = np.zeros(len(phen))
    #1. take route[0] to route[i-1] and add them in order to new_route
    new_route[0:i] = phen[0:i]
    print(phen[0:i])
    #2. take route[i] to route[k] and add them in reverse order to new_route
    new_route[i:k+1] = np.flip(phen[i:k+1])
    print(np.flip(phen[i:k+1]))
    #3. take route[k+1] to end and add them in order to new_route
    new_route[k+1:] = phen[k+1:]
    print(phen[k+1:])
    return new_route

print(two_opt_swap([0,1,2,3,4,5,6,7,0], 3, 6))

def two_opt_search(phen:List[int]) -> List[int]:
    fitness = eval_fitness(phen)
    found_better_phen = False
    while True:
        for i in range(len(phen)-1):
            for k in range(i+1, len(phen)):
                new_phen = two_opt_swap(phen, i, k)
                new_fitness = eval_fitness(new_phen)
                if new_fitness > fitness:
                    phen = new_phen
                    fitness = new_fitness
                    found_better_phen = True 
                    break
            if found_better_phen:
                found_better_phen = False
                break
        # End local search if no progress has been made
        if not found_better_phen:
            break


[0, 1, 2]
[6 5 4 3]
[7, 0]
[0. 1. 2. 6. 5. 4. 3. 7. 0.]


In [ ]:
def simple_EA():
    pass

def two_opt_MA():
    pass